In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.1'
spark_version = 'spark-3.4.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,274 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,466 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,420 kB]
Hit:13 https://ppa.launchpadcontent.net/

In [7]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [10]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

In [11]:
# 2. Create a temporary view of the DataFrame.
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep = ",", header = True)
df.createOrReplaceTempView("home_sales")
df

DataFrame[id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [13]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
price = spark.sql("""
SELECT YEAR(date) as year,
ROUND(AVG(CAST(price as DOUBLE)),2) as avg_price
FROM home_sales
WHERE bedrooms = '4'
GROUP BY year
ORDER By year
""")
price.show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [14]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
built = spark.sql("""
SELECT date_built,
ROUND(AVG(CAST(price as DOUBLE)),2) as avg_price
FROM home_sales
WHERE bedrooms = '3' AND bathrooms = '3'
GROUP BY date_built
ORDER By date_built
""")
built.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [20]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
a_lot = spark.sql("""
SELECT date_built, ROUND(AVG(CAST(price as DOUBLE)), 2) as avg_price
FROM home_sales
WHERE bedrooms = '3'
AND bathrooms = '3'
AND floors = '2'
AND CAST(sqft_living as INT) >= 2000
GROUP BY date_built
ORDER BY date_built
""")
a_lot.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [22]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
price = spark.sql("""
SELECT view, ROUND(AVG(CAST(price as DOUBLE)), 2) as avg_price
FROM home_sales
WHERE CAST(price as DOUBLE) >= 350000
GROUP BY view
ORDER BY view
""")

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.06288838386535645 seconds ---


In [23]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [24]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [25]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
import time

start_time = time.time()
price = spark.sql("""
SELECT view, ROUND(AVG(CAST(price as DOUBLE)), 2) as avg_price
FROM home_sales
WHERE CAST(price as DOUBLE) >= 350000
GROUP BY view
ORDER BY view
""")

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.03688955307006836 seconds ---


In [26]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
homes_sales.write.partitionBy("date_built").parquet()"I\am\not_sure_because_I_am_using_google_colab" # If it was on my c: drive it could be "C:\user\desktop\module-22\Start_Code\"

SyntaxError: ignored

In [ ]:
# 11. Read the parquet formatted data.
home_sales_2 = spark.read.parquet("C:\user\desktop\module-22\Start_Code\")

In [ ]:
# 12. Create a temporary table for the parquet data.
homes_sales_parquet.createOrReplaceTempView('homes_sales_parquet')

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.


# This won't run because I am not saving it to a location that it can load from, but this would be the code.
start_time = time.time()
price = spark.sql("""
SELECT view, ROUND(AVG(CAST(price as DOUBLE)), 2) as avg_price
FROM home_sales_parquet
WHERE CAST(price as DOUBLE) >= 350000
GROUP BY view
ORDER BY view
""")

print("--- %s seconds ---" % (time.time() - start_time))

--- 7.104873657226562e-05 seconds ---


In [27]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [28]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False